# Sequence to Sequence Modelling

**Note**: All code should be in Python3. Keras version should be 2.0.4. The directory structure on the bitbucket repo should be exactly same as the hw4.zip provided to you (with the exception of data directory. Do not upload it). To push the code to remote repo, use the same instructions as given in HW0. **Double check you remote repo for correct directory structure. We won't consider any regrade requests based on wrong directory structure penalty. Again, do not upload data to your bitbucket repo ** <br>
**The data provided to you should not be used for any other purpose than this course. You must not distribute it or upload it to any public platform.**

![title](seq2seq.png)

In this assignment, we are going to solve the problem of summarization using a sequence to sequence model. In a sequence to sequence problem, we have an encoder and a decoder. We feed the sequence of word embeddings to an encoder and train decoder to learn the summaries. We will be seeing 2 types of encoder decoder architectures in this assignment

# Preparing Inputs

The first part of the assignment is to prepare data. You are given training data in train_article.txt, in which each line is the first sentence from an article, and training summary sentences in train_title.txt, which are the corresponding titles of the article. You will be training the model to predict the title of an article given the first sentence of that article, where title generation is a summarization task. Let us limit the maximum vocabulary size to 20000 and maximum length of article to 200 (These are just initial params to get you started and we recommend experimenting, to improve your scores after you are done with your first implementation)

In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Embedding, LSTM, Input
from keras.models import Model
from attention_decoder import AttentionDecoder
from keras.models import load_model
from rouge import Rouge 
from rouge import FilesRouge
import codecs
import scipy
import datetime
import sys
import numpy as np

Using TensorFlow backend.


In [2]:
import keras; print( keras.__version__)

2.0.9


In [3]:
MAX_LEN = 200
VOCAB_SIZE = 20000
EXAMPLE_SIZE = 50000

Write a function which takes article file, summary file, maximum length of sentence and vocabulary size and does the following
* Create vocabulary: Take most frequent VOCAB_SIZE number of words from article file. Add two special symbols ZERO at start and UNK at end to finally have VOCAB_SIZE + 2 words. Use this array as idx2word. Repeat the process for summary data to create another idx2word corresponding to it. 
* Using the above idx2word for both article and summary data, create word2idx, which will map every word to its index in idx2word array. 
* Convert the words in the article and summary data to their corresponding index from word2idx. If a word is not present in the vocab, use the index of UNK. 
* After the above preprocessing, each sentence in article and summary data should be a list of indices
* Now find the max length of a sentence (which is basically number of indices in a sentence) in article data. Pad every sentence in article data to that length, so that all sentences are of same length. You may use pad_sequences function provided by keras. Do the same for title data.
* return the following outputs transformed article data, vocab size of article data, idx2word(articledata), word2idx(articledata),transformed summary data, vocab size of summary data, idx2word(summarydata), word2idx(summarydata)

In [4]:
def word2idx(idx2word_list):
    return {word: idx for (idx, word) in enumerate(idx2word_list)}

def read_file(file_path, length_limit, start=''):
    # read in file data
    data = []    
    f = None
    max_length = 1
    try:
        f = open(file_path, 'r')
        for line in f:
            length = len(line.split())
            if length > length_limit:
                continue
            max_length = np.max([max_length, length])
            data.append(start + line)
    except BaseException as e:
        print (str(e))
        sys.exit(1)
    finally:
        if f: f.close()

    return data, max_length

def idx2word(data_list, vocab_size):
    
    # build vocabulary
    word_ranking = dict()
    # flatten data
    tokens = [item for sublist in [x.split() for x in data_list] for item in sublist]
    vocab_set = set(tokens)
    for word in vocab_set:
        word_ranking[word] = tokens.count(word)
    assert len(word_ranking.keys()) == len(vocab_set)
    vector = ['ZERO'] + [k for k in sorted(word_ranking, key=word_ranking.get, 
                                      reverse=True)][:vocab_size] + ['UNK']
    assert len(vector) == len(set(vector))
    assert len(vector) <= vocab_size+2
    return data_list, vector, len(set(vector)), set(vector)

def encode_file_text(file_data, word2idx, max_length):
    encoded_text = []
    unknown_idx = word2idx.get('UNK')
    ndx = 0
    for line in file_data:
        encoded_sentence = []
        for w in line.split(' '):
            encoded_sentence.append(float(word2idx.get(w, unknown_idx)))
        encoded_text.append(encoded_sentence)
        ndx += 1
    return pad_sequences(encoded_text, maxlen=max_length)

def onehot_encode(encoded_summary_data, max_sequence_len, vocab_size):
    encoded = np.zeros([len(encoded_summary_data), max_sequence_len, vocab_size])
    for i, sentence in enumerate(encoded_summary_data):
        for j, word in enumerate(sentence):
            encoded[i, j, word] = 1.
    return encoded

def load_data(article, summary, max_len, vocab_size, train=1):
    # read in file data
    article_data, max_length_article = read_file(article, max_len)
    summary_data, max_length_summary = read_file(summary, max_len, start='<s>')

    article_data, idx2word_article, vocab_size_article, vocab_article = \
        idx2word(article_data, vocab_size)
    word2idx_article = word2idx(idx2word_article)
    summary_data, idx2word_summary, vocab_size_summary, vocab_summary = \
        idx2word(summary_data, vocab_size)
    word2idx_summary = word2idx(idx2word_summary)
    
    if train:
        max_length = np.max([max_length_summary, max_length_article])
    else:
        max_length = max_len
        
    encoded_article = encode_file_text(article_data, word2idx_article, max_length)
    encoded_summary = encode_file_text(summary_data, word2idx_summary, max_length)
    target_data = encode_file_text(shift_target(summary_data), word2idx_summary, max_length)

    return (encoded_article, vocab_article, vocab_size_article, idx2word_article, word2idx_article,
            encoded_summary, vocab_summary, vocab_size_summary, idx2word_summary, word2idx_summary, 
            target_data, max_length)

def shift_target(file_data):
    target = []
    for line in file_data:
        target.append(line[1:] + 'UNK')
    return target

def reverse_onehot_encoding(onehot_data, idx2word_lookup):
    string = ''
    for word_position in range(onehot_data.shape[0]):
        word = idx2word_lookup[np.argmax(onehot_data[word_position,:], axis=0)]
        string = string + word + ' '
    return string

Now use the above function to load the training data from article and summary (i.e. title) files. Do note that, based on your model architecture, you may need to further one-hot vectorize your input to the model

In [6]:
print (datetime.datetime.now())
encoded_article, vocab_article, vocab_size_article, idx2word_article, word2idx_article, \
    encoded_summary, vocab_summary, vocab_size_summary, idx2word_summary, word2idx_summary,\
    target_data, max_length = load_data('data/train_article.txt', 'data/train_title.txt', MAX_LEN, 
    VOCAB_SIZE)
print (datetime.datetime.now())

2017-12-15 00:42:21.705540
2017-12-15 00:51:40.701128


# Unidirectional LSTM Encoder Decoder 

Define the parameters for your LSTM encoder decoder model. 

Create a Unidirectional encoder decoder LSTM model in create_model function. The model should have a LSTM Unidirectional layer as encoder and a LSTM decoder.
Use categorical_cross_entropy loss and experiment with different optimizers to improve your score.

In [14]:
BATCH_SIZE = 100
NUM_LAYERS = 1
HIDDEN_DIM = 300
ITERATIONS = int(len(encoded_article)/BATCH_SIZE) 

In [9]:
print (datetime.datetime.now())
#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
f.close()
print('G Word embeddings:', len(gembeddings_index))

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((vocab_size_article, HIDDEN_DIM))
print ('Glove embedding matrix dims:', g_word_embedding_matrix.shape)

for word, i in word2idx_article.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
print (datetime.datetime.now())

2017-12-14 06:46:13.757222
G Word embeddings: 1917494
Glove embedding matrix dims: (20002, 300)
2017-12-14 06:49:33.675056


In [11]:
def create_UniLSTM(X_vocab_len, X_max_len, y_vocab_len, y_max_len, hidden_size, num_layers):
    # create and return the model for unidirectional LSTM encoder decoder
    embedding = Embedding(X_vocab_len, hidden_size, input_length=X_max_len, trainable=False, 
        weights=[g_word_embedding_matrix])
    encoder_input = Input(shape=(X_max_len,), dtype='float32')
    embedded_sequences = embedding(encoder_input)
    _, state1, state1 = LSTM(hidden_size, input_shape=(X_max_len, 
                                       X_vocab_len), return_state=True)(embedded_sequences)
    decoder_input = Input(shape=(None, y_vocab_len))
    decoder_outputs, _, _ = LSTM(hidden_size, return_sequences=True, 
                                 return_state=True)(decoder_input, initial_state=[state1, state1])
    output = Dense(y_vocab_len, activation='softmax')(decoder_outputs)
    return Model([encoder_input, decoder_input], output)

# Train the Model

Now that we have everything in place, we can run our model. We recommend training the model in batches instead of training on all 50,000 article-title pairs at once, if you encounter memory contraints

In [12]:
print (datetime.datetime.now()) # 1 second
try:
    model = create_UniLSTM(vocab_size_article, max_length, vocab_size_summary, max_length, HIDDEN_DIM, NUM_LAYERS)
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    model.summary()
finally:
    print(datetime.datetime.now())

2017-12-14 06:50:21.104095
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 300)      6000600     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 20002)  0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 300), (None, 721200      embedding_1[0][0]                
__________________________________________________________________________________

In [15]:
print('Train...') 
for i in range(ITERATIONS):
    X = encoded_article[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
    y = onehot_encode(encoded_summary[i*BATCH_SIZE:(i+1)*BATCH_SIZE], max_length, vocab_size_summary)
    t = onehot_encode(target_data[i*BATCH_SIZE:(i+1)*BATCH_SIZE], max_length, vocab_size_summary)
    model.fit([X, y], t,
          batch_size=BATCH_SIZE,
          epochs=5,
          validation_split=0.2,
          verbose = 1,
          shuffle = True)

Train...
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 3s 37ms/step - loss: 9.9043 - acc: 0.0000e+00 - val_loss: 9.7834 - val_acc: 0.7734
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 9.7877 - acc: 0.7656 - val_loss: 8.3832 - val_acc: 0.7734
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 8.3505 - acc: 0.7656 - val_loss: 6.7618 - val_acc: 0.7734
Epoch 4/5
80/80 [==============================] - 2s 21ms/step - loss: 6.6135 - acc: 0.7656 - val_loss: 5.0756 - val_acc: 0.7734
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 4.7939 - acc: 0.7656 - val_loss: 3.6731 - val_acc: 0.7734
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 3.8848 - acc: 0.7375 - val_loss: 2.5943 - val_acc: 0.7969
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 2.8732 - acc: 0.7375 - val_loss: 2.1011 - val_a

Epoch 5/5
80/80 [==============================] - 2s 21ms/step - loss: 1.4544 - acc: 0.7812 - val_loss: 1.9016 - val_acc: 0.7516
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 21ms/step - loss: 1.8088 - acc: 0.7668 - val_loss: 1.7012 - val_acc: 0.7750
Epoch 2/5
80/80 [==============================] - 2s 21ms/step - loss: 1.7539 - acc: 0.7668 - val_loss: 1.6400 - val_acc: 0.7750
Epoch 3/5
80/80 [==============================] - 2s 21ms/step - loss: 1.6509 - acc: 0.7668 - val_loss: 1.6152 - val_acc: 0.7750
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 1.5795 - acc: 0.7668 - val_loss: 1.5989 - val_acc: 0.7750
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 1.5394 - acc: 0.7668 - val_loss: 1.6636 - val_acc: 0.7750
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 1.7421 - acc: 0.7750 - val_loss: 1.7355 - val_acc: 0.7797
Ep

Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 1.6668 - acc: 0.7883 - val_loss: 1.9951 - val_acc: 0.7641
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 1.6291 - acc: 0.7879 - val_loss: 1.9862 - val_acc: 0.7656
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 1.5953 - acc: 0.8066 - val_loss: 2.2572 - val_acc: 0.7312
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 1.8577 - acc: 0.7668 - val_loss: 2.0825 - val_acc: 0.7203
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 1.8497 - acc: 0.7188 - val_loss: 1.9548 - val_acc: 0.7609
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 1.5541 - acc: 0.8000 - val_loss: 1.8423 - val_acc: 0.7547
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 1.4871 - acc: 0.8074 - val_loss: 1.9075 - val_acc: 0.7609
Train on 80 samples, validate on 20 samples
Ep

Epoch 3/5
80/80 [==============================] - 2s 21ms/step - loss: 1.2962 - acc: 0.8207 - val_loss: 1.3583 - val_acc: 0.8203
Epoch 4/5
80/80 [==============================] - 2s 21ms/step - loss: 1.2791 - acc: 0.8180 - val_loss: 1.3760 - val_acc: 0.8313
Epoch 5/5
80/80 [==============================] - 2s 21ms/step - loss: 1.2743 - acc: 0.8188 - val_loss: 1.3822 - val_acc: 0.8188
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 1.4342 - acc: 0.8094 - val_loss: 1.6476 - val_acc: 0.7937
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 1.4371 - acc: 0.8145 - val_loss: 1.8237 - val_acc: 0.7984
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 1.6483 - acc: 0.8160 - val_loss: 1.6708 - val_acc: 0.7937
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 1.4159 - acc: 0.8105 - val_loss: 1.5528 - val_acc: 0.8031
Epoch 5/5
80/80 [=============================

Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 1.1816 - acc: 0.8281 - val_loss: 1.5644 - val_acc: 0.7984
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 1.1424 - acc: 0.8324 - val_loss: 1.5205 - val_acc: 0.8078
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 1.1169 - acc: 0.8383 - val_loss: 1.5350 - val_acc: 0.7984
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 1.0969 - acc: 0.8379 - val_loss: 1.5398 - val_acc: 0.8078
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 1.4363 - acc: 0.8195 - val_loss: 1.5201 - val_acc: 0.7937
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 1.3885 - acc: 0.8152 - val_loss: 1.5118 - val_acc: 0.7969
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 1.3442 - acc: 0.8199 - val_loss: 1.5040 - val_acc: 0.7953
Epoch 4/5
80/80 [=============================

...

Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 0.7961 - acc: 0.9082 - val_loss: 0.6401 - val_acc: 0.9328
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 0.7758 - acc: 0.9094 - val_loss: 0.6551 - val_acc: 0.9297
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 0.6800 - acc: 0.9312 - val_loss: 0.5142 - val_acc: 0.9500
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 0.6309 - acc: 0.9336 - val_loss: 0.5164 - val_acc: 0.9516
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 0.6018 - acc: 0.9355 - val_loss: 0.5163 - val_acc: 0.9484
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5784 - acc: 0.9371 - val_loss: 0.5202 - val_acc: 0.9484
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5612 - acc: 0.9387 - val_loss: 0.5265 - val_acc: 0.9469
Train on 80 samples, validate on 20 samples
Ep

Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5607 - acc: 0.9375 - val_loss: 0.5800 - val_acc: 0.9375
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5447 - acc: 0.9402 - val_loss: 0.5590 - val_acc: 0.9375
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5345 - acc: 0.9375 - val_loss: 0.5842 - val_acc: 0.9344
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 0.6829 - acc: 0.9285 - val_loss: 0.7173 - val_acc: 0.9234
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 0.6390 - acc: 0.9309 - val_loss: 0.7198 - val_acc: 0.9234
Epoch 3/5
80/80 [==============================] - 2s 21ms/step - loss: 0.6121 - acc: 0.9328 - val_loss: 0.7159 - val_acc: 0.9234
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5907 - acc: 0.9352 - val_loss: 0.7257 - val_acc: 0.9250
Epoch 5/5
80/80 [=============================

Epoch 2/5
80/80 [==============================] - 2s 21ms/step - loss: 0.3420 - acc: 0.9605 - val_loss: 0.4332 - val_acc: 0.9531
Epoch 3/5
80/80 [==============================] - 2s 21ms/step - loss: 0.3208 - acc: 0.9605 - val_loss: 0.4270 - val_acc: 0.9531
Epoch 4/5
80/80 [==============================] - 2s 20ms/step - loss: 0.3050 - acc: 0.9617 - val_loss: 0.4363 - val_acc: 0.9531
Epoch 5/5
80/80 [==============================] - 2s 20ms/step - loss: 0.2917 - acc: 0.9633 - val_loss: 0.4300 - val_acc: 0.9531
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 21ms/step - loss: 0.4812 - acc: 0.9465 - val_loss: 0.3718 - val_acc: 0.9625
Epoch 2/5
80/80 [==============================] - 2s 21ms/step - loss: 0.4463 - acc: 0.9500 - val_loss: 0.3633 - val_acc: 0.9641
Epoch 3/5
80/80 [==============================] - 2s 21ms/step - loss: 0.4208 - acc: 0.9504 - val_loss: 0.3687 - val_acc: 0.9625
Epoch 4/5
80/80 [=============================

80/80 [==============================] - 2s 20ms/step - loss: 0.6001 - acc: 0.9363 - val_loss: 0.4254 - val_acc: 0.9594
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5520 - acc: 0.9395 - val_loss: 0.4286 - val_acc: 0.9547
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5243 - acc: 0.9402 - val_loss: 0.4279 - val_acc: 0.9563
Epoch 4/5
80/80 [==============================] - 2s 21ms/step - loss: 0.5043 - acc: 0.9406 - val_loss: 0.4474 - val_acc: 0.9547
Epoch 5/5
80/80 [==============================] - 2s 21ms/step - loss: 0.4948 - acc: 0.9430 - val_loss: 0.4253 - val_acc: 0.9547
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 21ms/step - loss: 0.6413 - acc: 0.9320 - val_loss: 0.7105 - val_acc: 0.9250
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5943 - acc: 0.9387 - val_loss: 0.7140 - val_acc: 0.9234
Epoch 3/5
80/80 [==============================] - 2s 20

Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5882 - acc: 0.9367 - val_loss: 0.6884 - val_acc: 0.9312
Epoch 2/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5492 - acc: 0.9402 - val_loss: 0.6797 - val_acc: 0.9312
Epoch 3/5
80/80 [==============================] - 2s 20ms/step - loss: 0.5182 - acc: 0.9418 - val_loss: 0.6868 - val_acc: 0.9297
Epoch 4/5
80/80 [==============================] - 2s 21ms/step - loss: 0.4982 - acc: 0.9449 - val_loss: 0.6869 - val_acc: 0.9312
Epoch 5/5
80/80 [==============================] - 2s 21ms/step - loss: 0.4809 - acc: 0.9465 - val_loss: 0.6974 - val_acc: 0.9281
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 21ms/step - loss: 0.6339 - acc: 0.9344 - val_loss: 0.7107 - val_acc: 0.9281
Epoch 2/5
80/80 [==============================] - 2s 21ms/step - loss: 0.5931 - acc: 0.9363 - val_loss: 0.7066 - val_acc: 0.9281
Ep

In [16]:
model.save('model_wo_attention.h5')

/usr/local/lib/python3.4/dist-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [53]:
model = load_model('model_wo_attention.h5')

# Evaluation using Rouge score 

Now that you have trained the model, load the test data i.e. test_article.txt and corresponding reference titles test_title.txt. Process test_article.txt in the same way as you did your train_article.txt. Then use your model to predict the titles.
When you have your model predicted titles, and the reference titles (test_title.txt) calculate the Rouge score corresponding to your predictions. <br>
You should install rouge by executing "pip3 install rouge". Refer https://pypi.python.org/pypi/rouge/0.2.1 for documentation on how to use the package.

In [47]:
print (datetime.datetime.now())
encoded_article_test, vocab_article_test, vocab_size_article_test, idx2word_article_test, word2idx_article_test, \
encoded_summary_test, vocab_summary_test, vocab_size_summary_test, idx2word_summary_test, word2idx_summary_test, \
target_data_test, max_length_test = load_data('data/test_article.txt', 'data/test_title.txt', max_length, VOCAB_SIZE, train=0)
print (datetime.datetime.now())

2017-12-14 08:21:52.085717
2017-12-14 08:22:20.042880


In [48]:
print (datetime.datetime.now())
try:
    f = open('wo_attention_predictions.txt', 'w+')
    ITERATIONS = int(len(encoded_article_test)/BATCH_SIZE)
    for i in range(ITERATIONS):
        X_test = encoded_article_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        y_test = onehot_encode(encoded_summary_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE], max_length, vocab_size_summary)
        y_hat = model.predict([X_test, y_test], batch_size=BATCH_SIZE, verbose=1)
        hyp = [reverse_onehot_encoding(x, idx2word_summary) for x in y_hat]
        for h in hyp:
            f.write(h + '\n')
# except:
#     print (encoded_summary[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
finally:
    if f: f.close()
        
print (datetime.datetime.now())

2017-12-14 08:22:37.734390
100/100 [==============================] - 1s 8ms/step
2017-12-14 08:23:46.701118


In [49]:
files_rouge = FilesRouge('wo_attention_predictions.txt', 'data/test_title.txt')
print (files_rouge.get_scores(avg=True))

{'rouge-l': {'r': 0.072881065760037403, 'f': 0.01862120765777453, 'p': 0.017727272727272727}, 'rouge-2': {'r': 0.0024844475552458748, 'f': 0.0020196680705159564, 'p': 0.0017095859611173632}, 'rouge-1': {'r': 0.074692252201243745, 'f': 0.067554868962038037, 'p': 0.061987836187720094}}


# Tensorboard Visualization 

We recommended training the data in batches because of our tensor constraints. This also presents us with a challenge of visualizing loss function and accuracy change with each epoch. Keras has an inbuilt function called fit_generator which takes in a generator function and gives the required batch for training. Use this Function to load data in batches of 100 for 200 steps_per_epoch. Run the training for 10 epochs. Use Keras callbacks to send data to tensorboad (you can look this up online). 

Once your training is done. Go to command line and run tensorboard. By default Tensorboard opens on 6006 port. Do remember to allow traffic on the same for gcloud (like you did for previous assignment). You can see various metrics depending on what you want to track like loss, accuracy, validation loss and validation accuracy over epochs. Attach the plots of loss and accuracy from the tensorboard display in the notebook

In [ ]:
# TO-DO

# Unidirectional LSTM Encoder Decoder With Attention 

Define the parameters for your LSTM encoder decoder model with attention

In [11]:
BATCH_SIZE = 100
VOCAB_SIZE = 10000
NUM_LAYERS = 1
HIDDEN_DIM = 128
ITERATIONS = int(EXAMPLE_SIZE/BATCH_SIZE)

In [ ]:
print (datetime.datetime.now())
encoded_article, vocab_article, vocab_size_article, idx2word_article, word2idx_article, \
    encoded_summary, vocab_summary, vocab_size_summary, idx2word_summary, word2idx_summary,\
    target_data, max_length = load_data('data/train_article.txt', 'data/train_title.txt', MAX_LEN, 
    VOCAB_SIZE)
print (datetime.datetime.now())

2017-12-15 04:22:41.849353
2017-12-15 04:31:27.786777


You would've observed that the summaries are not yet perfect. This is because in encoder decoder architecture, only the final state of encoder is used to calculate the probabilities. We now move to a more general approach called attention based approach. In this, we take a weighted sum of all weights of encoder instead of just the last one. You are already provided an attention_decoder.py file with AttentionDecoder. Add this layer on top of your encoder and run the same experiment as before. For this part, you don't need to worry about return_probabilities argument to create_UniLSTMwithAttention function. Just pass it as an argument to your attention decoder layer. When return_probabilities is false, the attention decoder returns prediction model, which is what you need for this part of the assignment. When return_probabilities is true, the attention decoder returns the probability model, which you will be using later in the Analysis part of this assignment

In [6]:
def create_UniLSTMwithAttention(X_vocab_len, X_max_len, y_vocab_len, y_max_len, hidden_size, num_layers, 
        return_probabilities = False):
    # create and return the model for unidirectional LSTM encoder decoder with attention
    model = Sequential()
    model.add(Embedding(X_vocab_len, hidden_size, input_length=X_max_len, trainable=True))
    model.add(LSTM(hidden_size, input_shape=(X_max_len, X_vocab_len), return_sequences=True))
    model.add(AttentionDecoder(hidden_size, output_dim=y_vocab_len, return_probabilities=return_probabilities))
    return model

In [11]:
print (datetime.datetime.now()) # 1 hour
try:
    model = create_UniLSTMwithAttention(vocab_size_article, max_length, vocab_size_summary, max_length, HIDDEN_DIM, NUM_LAYERS)
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    model.summary()
finally:
    print(datetime.datetime.now())

2017-12-15 01:10:42.080526
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           1280256   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
AttentionDecoder (AttentionD (None, 32, 10002)         106599382 
Total params: 108,011,222
Trainable params: 108,011,222
Non-trainable params: 0
_________________________________________________________________
2017-12-15 02:08:44.575416


# Train the Model

Train the model, as you did before, for the model without attention

In [12]:
print('Train...')
for i in range(ITERATIONS):
    X = encoded_article[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
    y = onehot_encode(encoded_summary[i*BATCH_SIZE:(i+1)*BATCH_SIZE], max_length, vocab_size_summary)
    model.fit(X, y,
          batch_size=BATCH_SIZE,
          epochs=5,
          validation_split=0.2,
          verbose = 1,
          shuffle = True)

Train...
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 22s 278ms/step - loss: 9.2110 - acc: 0.0000e+00 - val_loss: 9.1751 - val_acc: 0.7703
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 9.1735 - acc: 0.7641 - val_loss: 9.0481 - val_acc: 0.7422
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 9.0336 - acc: 0.7348 - val_loss: 8.2716 - val_acc: 0.7422
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 8.1696 - acc: 0.7344 - val_loss: 6.5028 - val_acc: 0.7422
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 6.1935 - acc: 0.7344 - val_loss: 5.3486 - val_acc: 0.7734
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 5.4579 - acc: 0.7375 - val_loss: 4.3174 - val_acc: 0.7672
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 4.4300 - acc: 0.7070 - val_loss: 3.5165 - val

Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.7572 - acc: 0.7812 - val_loss: 2.1529 - val_acc: 0.7516
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 2.0720 - acc: 0.7668 - val_loss: 1.8482 - val_acc: 0.7750
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.9752 - acc: 0.7668 - val_loss: 1.8830 - val_acc: 0.7750
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.9475 - acc: 0.7668 - val_loss: 1.9559 - val_acc: 0.7750
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.9935 - acc: 0.7668 - val_loss: 1.8599 - val_acc: 0.7750
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.8485 - acc: 0.7668 - val_loss: 1.8577 - val_acc: 0.7750
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 2.0090 - acc: 0.7750 - val_loss: 2.0712 - val_acc: 0.7797
Ep

Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 2.0500 - acc: 0.7496 - val_loss: 2.5108 - val_acc: 0.7266
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 2.1402 - acc: 0.7496 - val_loss: 2.3932 - val_acc: 0.7266
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 2.0094 - acc: 0.7664 - val_loss: 2.2063 - val_acc: 0.7281
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.8898 - acc: 0.7664 - val_loss: 2.1729 - val_acc: 0.7281
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.8010 - acc: 0.7664 - val_loss: 2.1153 - val_acc: 0.7281
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.7459 - acc: 0.7664 - val_loss: 2.4929 - val_acc: 0.7281
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 2.0010 - acc: 0.7664 - val_loss: 2.6159 - val_acc: 0.7281
Train on 80 samples, validate on 20 samples
Ep

Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5045 - acc: 0.7805 - val_loss: 1.6018 - val_acc: 0.7891
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4787 - acc: 0.7805 - val_loss: 1.7192 - val_acc: 0.7891
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5800 - acc: 0.7805 - val_loss: 1.7651 - val_acc: 0.7891
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.8704 - acc: 0.7762 - val_loss: 2.1638 - val_acc: 0.7609
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 2.0315 - acc: 0.7762 - val_loss: 1.9595 - val_acc: 0.7609
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.7690 - acc: 0.7762 - val_loss: 1.8928 - val_acc: 0.7609
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6746 - acc: 0.7762 - val_loss: 1.8397 - val_acc: 0.7609
Epoch 5/5
80/80 [=============================

Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6734 - acc: 0.7961 - val_loss: 2.0221 - val_acc: 0.7563
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6012 - acc: 0.7961 - val_loss: 1.9702 - val_acc: 0.7563
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5272 - acc: 0.7961 - val_loss: 1.9316 - val_acc: 0.7563
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4606 - acc: 0.7961 - val_loss: 1.8740 - val_acc: 0.7563
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.7050 - acc: 0.7746 - val_loss: 1.6939 - val_acc: 0.7531
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6548 - acc: 0.7746 - val_loss: 1.8592 - val_acc: 0.7531
Epoch 3/5
80/80 [==============================] - 3s 31ms/step - loss: 1.8314 - acc: 0.7746 - val_loss: 2.0228 - val_acc: 0.7531
Epoch 4/5
80/80 [=============================

...

Epoch 4/5
80/80 [==============================] - 3s 31ms/step - loss: 1.9214 - acc: 0.7523 - val_loss: 1.7177 - val_acc: 0.7812
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.9185 - acc: 0.7535 - val_loss: 1.7201 - val_acc: 0.7766
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6190 - acc: 0.7910 - val_loss: 1.4094 - val_acc: 0.8313
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5849 - acc: 0.7977 - val_loss: 1.4073 - val_acc: 0.8281
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5490 - acc: 0.7969 - val_loss: 1.3833 - val_acc: 0.8313
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5344 - acc: 0.7977 - val_loss: 1.3950 - val_acc: 0.8297
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5252 - acc: 0.7961 - val_loss: 1.3841 - val_acc: 0.8313
Train on 80 samples, validate on 20 samples
Ep

Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5615 - acc: 0.8004 - val_loss: 1.5600 - val_acc: 0.7984
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5616 - acc: 0.8012 - val_loss: 1.5558 - val_acc: 0.7984
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5478 - acc: 0.7992 - val_loss: 1.5716 - val_acc: 0.7969
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5552 - acc: 0.8062 - val_loss: 1.7787 - val_acc: 0.7781
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5168 - acc: 0.8059 - val_loss: 1.8185 - val_acc: 0.7734
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5152 - acc: 0.8055 - val_loss: 1.7869 - val_acc: 0.7781
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5048 - acc: 0.8059 - val_loss: 1.8303 - val_acc: 0.7734
Epoch 5/5
80/80 [=============================

Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.2028 - acc: 0.8270 - val_loss: 1.3435 - val_acc: 0.8047
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.1536 - acc: 0.8293 - val_loss: 1.4136 - val_acc: 0.8109
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.1422 - acc: 0.8375 - val_loss: 1.3376 - val_acc: 0.8078
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.1263 - acc: 0.8316 - val_loss: 1.4066 - val_acc: 0.8094
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.2893 - acc: 0.8305 - val_loss: 1.1073 - val_acc: 0.8422
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.2507 - acc: 0.8301 - val_loss: 1.0787 - val_acc: 0.8484
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.2429 - acc: 0.8340 - val_loss: 1.0924 - val_acc: 0.8391
Epoch 4/5
80/80 [=============================

80/80 [==============================] - 2s 31ms/step - loss: 1.5614 - acc: 0.8047 - val_loss: 1.2345 - val_acc: 0.8594
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5221 - acc: 0.8137 - val_loss: 1.2528 - val_acc: 0.8500
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4891 - acc: 0.8117 - val_loss: 1.2201 - val_acc: 0.8547
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4762 - acc: 0.8133 - val_loss: 1.2486 - val_acc: 0.8484
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4676 - acc: 0.8117 - val_loss: 1.2183 - val_acc: 0.8594
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6387 - acc: 0.7887 - val_loss: 1.7249 - val_acc: 0.7797
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.6289 - acc: 0.7914 - val_loss: 1.7589 - val_acc: 0.7812
Epoch 3/5
80/80 [==============================] - 2s 31

Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5015 - acc: 0.8148 - val_loss: 1.7162 - val_acc: 0.7953
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4485 - acc: 0.8160 - val_loss: 1.7466 - val_acc: 0.7953
Epoch 3/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4503 - acc: 0.8180 - val_loss: 1.7174 - val_acc: 0.7953
Epoch 4/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4288 - acc: 0.8160 - val_loss: 1.7476 - val_acc: 0.7953
Epoch 5/5
80/80 [==============================] - 2s 31ms/step - loss: 1.4341 - acc: 0.8180 - val_loss: 1.7171 - val_acc: 0.7953
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5375 - acc: 0.8055 - val_loss: 1.6362 - val_acc: 0.8047
Epoch 2/5
80/80 [==============================] - 2s 31ms/step - loss: 1.5348 - acc: 0.8031 - val_loss: 1.6170 - val_acc: 0.8062
Ep

In [13]:
model.save_weights('model_w_attention.h5')

# Evaluation using Rouge Score

Evaluate your model as before, using Rouge score. Ideally, your scores for the model with attention should be better than the model without attention

In [ ]:
print (datetime.datetime.now())
encoded_article_test, vocab_article_test, vocab_size_article_test, idx2word_article_test, word2idx_article_test, \
encoded_summary_test, vocab_summary_test, vocab_size_summary_test, idx2word_summary_test, word2idx_summary_test, \
target_data_test, max_length_test = load_data('data/test_article.txt', 'data/test_title.txt', max_length, VOCAB_SIZE, train=0)
print (datetime.datetime.now())

In [16]:
print (datetime.datetime.now())
try:
    f = open('attention_predictions.txt', 'w+')
    ITERATIONS = int(len(encoded_article_test)/BATCH_SIZE)
    for i in range(ITERATIONS):
        X_test = encoded_article_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        y_hat = model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)
        hyp = [reverse_onehot_encoding(x, idx2word_summary) for x in y_hat]
        for h in hyp:
            f.write(h + '\n')
finally:
    if f: f.close()
        
print (datetime.datetime.now())

2017-12-15 03:54:31.525130
100/100 [==============================] - 1s 7ms/step
2017-12-15 03:55:19.785006


In [17]:
files_rouge = FilesRouge('attention_predictions.txt', 'data/test_title.txt')
print (files_rouge.get_scores(avg=True))

{'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0004125345244203523, 'p': 0.00039393939393939391, 'r': 0.00167864516435945}, 'rouge-1': {'f': 0.0021959736346041305, 'p': 0.0032499999999999999, 'r': 0.0016960024102881245}}


# Perplexity 

Even though we evaluate our models on ROUGE score, we don't train our neural networks to learn better ROUGE score for the fact that ROUGE score is a complicated nonconvex function. How does our model learn then? In information theory, Perplexity is a measure of how good a model is.

Perplexity                     $$ = 2^{{-\sum _{x}p(x)\log _{2}p(x)}}$$ 
            
Lower the perplexity, better the model. Justify why our model learns well with our loss function? 

We are trying to minimize our loss function, which is the cross entropy $H(p, p)$. But the perplexity equals $2$ raised to the power of the cross entropy, i.e. $ 2^{{-\sum _{x}p(x)\log _{2}p(x)}} = 2^{H(p, p)}$. So they are proportional: lowering the loss function lowers the perplexity.

# Analysis

You will now plot the attention weights for a sentence and it's output. If a grid cell is white in the plot, it means that during summary, the word on x-axis corresponds to the word on y-axis. You are provided with a Visualizer class for helping you out. Make sure you install matplotlib using sudo pip3 install matplotlib and also install python3-tk using sudo apt-get install python3-tk

In [7]:
import argparse
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

In [8]:
class Visualizer(object):

    def __init__(self):
        """
            Visualizes attention maps
        """
        
        
    def set_models(self, pred_model, proba_model):
        """
            Sets the models to use
            :param pred_model: the prediction model
            :param proba_model: the model that outputs the activation maps
        """
        self.pred_model = pred_model
        self.proba_model = proba_model

    def attention_map(self, text, padded_data_vec, y_idx_to_word):
        """
            Displays the attention weights graph
            param: input sentence
            param: padded_data_vector for prediction
            param: idx2word dictionary for titles
        """
        input_length = len(text.split())
        
        # get the output sequence
        prediction = np.argmax(pred_model.predict(padded_data_vec), axis=2)[0]
        text_ = text.split()
        valids = [y_idx_to_word[index] for index in prediction if index > 0]
        sequence = ' '.join(valids)
        predicted_text = sequence.split()
        output_length = len(predicted_text)
        #get the weights
        activation_map = np.squeeze(self.proba_model.predict(padded_data_vec))[
            0:output_length, 0:input_length]
        
        plt.clf()
        f = plt.figure(figsize=(8, 8.5))
        ax = f.add_subplot(1, 1, 1)

        # add image
        i = ax.imshow(activation_map, interpolation='nearest', cmap='gray')
        
        # add colorbar
        cbaxes = f.add_axes([0.2, 0, 0.6, 0.03])
        cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
        cbar.ax.set_xlabel('Probability', labelpad=2)

        # add labels
        ax.set_yticks(range(output_length))
        ax.set_yticklabels(predicted_text[:output_length])
        
        ax.set_xticks(range(input_length))
        ax.set_xticklabels(text_[:input_length], rotation=45)
        
        ax.set_xlabel('Input Sequence')
        ax.set_ylabel('Output Sequence')

        # add grid and legend
        ax.grid()
        
        f.show()

You can initialize Visualizer class as follows

In [9]:
viz = Visualizer()

Visualizer has two methods.
- set_models 
- attention_map

The set_models takes in prediction model and probability model as inputs. In *create_UniLSTMwithAttention*, the model with *return_probabilities = False* which you already used in the training is the prediction model. For initializing probability model, call *create_UniLSTMwithAttention* with *return_probabilities = True* and initialize the weights with weights of prediction model. Now you can call set_models in this manner:

In [ ]:
print (datetime.datetime.now()) 
try:
    prob_model = create_UniLSTMwithAttention(vocab_size_article, max_length, vocab_size_summary, max_length, HIDDEN_DIM, 
                                     NUM_LAYERS, return_probabilities=True)
    prob_model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    prob_model.summary()
    
finally:
    print(datetime.datetime.now())

2017-12-15 04:31:27.795303


In [ ]:
prob_model.load_weights('model_w_attention.h5')

In [ ]:
pred_model = model
viz.set_models(pred_model,prob_model)

attention_map creates the weights map for you. You need to give a sample sentence, a test_data_vector on which we call call model.predict and your output idx2word dictionary. You can call it as follows

In [ ]:
#viz.attention_map(text,test_data_vector,idx2word)

Use the above Visualizer to visualize attention weights for 15 sentences, as instructed in the Analysis section of the accompanying HW document

In [ ]:
# viz.attention_map("london share prices up at midday", X_test[0,:,:], idx2word_summary)
# viz.attention_map("london share prices up at midday", X_test[0,:,:], idx2word_summary)

# Unidirectional LSTM Encoder Decoder With Attention and Beam Search (Extra Credit)

The models that you implemented till now had greedy decoder. Now implement a Decoder with Beam Search and show improved results